<a href="https://colab.research.google.com/github/olanigan/whisper-scribe/blob/main/WhisperTranscribe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q torch pytube transformers huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 71.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 101.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.3 MB/s eta 0:00:00


In [4]:
# import gradio as gr
import pytube as pt
from transformers import pipeline
from huggingface_hub import model_info
import torch

MODEL_NAME = "tarteel-ai/whisper-tiny-ar-quran" #this always needs to stay in line 8 :D sorry for the hackiness
lang = "ar"

device = 0 if torch.cuda.is_available() else "cpu"
pipe = pipeline(
    task="automatic-speech-recognition",
    model=MODEL_NAME,
    chunk_length_s=30,
    device=device,
)

In [6]:
def yt_transcribe(yt_url):
    yt = pt.YouTube(yt_url)
    # html_embed_str = _return_yt_html_embed(yt_url)
    stream = yt.streams.filter(only_audio=True)[0]
    stream.download(filename="audio.mp3")

    text = pipe("audio.mp3")["text"]

    return text

In [7]:
%%time
notes = yt_transcribe("https://www.youtube.com/watch?v=5tsbWHgBYpw&ab_channel=AlHadyAlNabawy")

CPU times: user 7min 35s, sys: 17.4 s, total: 7min 52s
Wall time: 8min 3s


In [10]:
notes

'وَحُوحٍ تُحْوِيهِمْ وَحُوحٍ تُحْوِيهِمْ وَحُوحٍ تُحْوِيهِمْ وَكِفُتْهُ مِنْ لَعَنِ الْحُدْلِ لَهُ مِنْ اللَّهِ وَسَلُّوا بَرَكْلَنِ مِنْ رَسَلُّ بَرَكْلَنِ مِنْ رَحْمَةٍ عَلَى آلِهِ غَصَحْتِهِ أَشْمَعِنَ اللَّهْلَ إِلَّا مَا جَأْتَهْنَ وَأَنْتَ تَزْمَعِدَ شَدَ اللَّهُمَ أَعِنْ عَلَى ذِكْرِكَ وَشُكْرِكَ وَحِسْنِ عِبَادِكَ رَبُكْرِ مِنْهِ مِنِ مِنِ مُسْنَ مَعِكْرِكْرِكْرِكْنَ وَحُمُ اللَّهُمُ الْمُ الْمُ الْمُ الْمُ الْمُ الْمُ الْمُ الْمُ الْمُ الْمُ اللَّهِمُ اللَّهِمُ اللَّهِمُ اللَّهِمُ اللَّهِ وَحُمُ اللَّهِ وَحُمُ اللَّهِ وَحُمَ اللَّهُمَا أَعِنْ عَلَى ذِكْرِكَ وَشُكْرِكَ وَحِسْمِ عِبَادِكَ رَبُكَ لِمُسْلَمَ لِكُمَحُتُ اللَّهُ بَكَثَةً إِسَخْلِفَتَ اسْتِكْسِي أُهْيَهُ أَأَحْدِي لَهُمْ مَا قَدْ جِسْرُ الْفَبْسُهُ أَطْسِي عَنْ عِلْمَهُمْ فَانَيْنِ وَدِيسْلِهِ تَنْتُبْسِي تَكْطِسُسَهُ أَا أُوِ اسْقَمْدِيهَا تَفَسْتُهُمْ لَسَنَّ بُسْلِنَّ أَوَاكْنِيُّهُمْ فَانِي ويشِيسَ أَوِ اسْقَمْدِيَّةَ فَاسْتَغْفَى عَامٌ لَسَنْ رَبُسْ فِيهِ نَأْوَاكَنِّي فَعَنِي بِوَدِيسٍ سَسُجُعَمْ فَأَنْزُلُ عَن

In [ ]:

pipe = pipeline(
    task="automatic-speech-recognition",
    model=MODEL_NAME,
    chunk_length_s=30,
    device=device,
)

In [11]:
MODEL_NAME = "openai/whisper-large-v2"
BATCH_SIZE = 8
FILE_LIMIT_MB = 1000
YT_LENGTH_LIMIT_S = 3600  # limit to 1 hour YouTube files

whisper = pipeline(
    task="automatic-speech-recognition",
    model=MODEL_NAME,
    chunk_length_s=30,
    device=device,
)

In [14]:
from transformers.pipelines.audio_utils import ffmpeg_read

def transcribe():
    with open("audio.mp3", "rb") as f:
        inputs = f.read()

    inputs = ffmpeg_read(inputs, pipe.feature_extractor.sampling_rate)
    inputs = {"array": inputs, "sampling_rate": pipe.feature_extractor.sampling_rate}

    text = whisper(inputs, batch_size=BATCH_SIZE, generate_kwargs={"task": "transcribe"}, return_timestamps=True)["text"]

    return text

In [15]:
%%time
notes2 = transcribe()

CPU times: user 9min 13s, sys: 14.8 s, total: 9min 28s
Wall time: 9min 33s


In [23]:
notes2[-300:]

' of course we will release the times. Okay guys? Barakallahu feekum, wajazakumullahu khair, wa subhanakallahumma bihamdika ashhadu an la ilaha illa anta wa astaghfirukallahu wa atubu ilayk, wasalamu alaikum wa rahmatullahi wa barakatuh. Get into your lines, straighten those lines. Salah starts in...'

In [18]:
def writeFile(text, filename):
  """Exports the given text to the given file."""
  with open(filename, 'w') as f:
    f.write(text)

In [22]:
# writeFile(notes2,"full.txt")
notes2

" اوه اوكي فولكس بسم الله الرحمن الرحيم الحمد لله رب العالمين اللهم صلي وسلم وبركة على نبينا محمد وعلى آله وصحبه اجمعين اللهم لا سهل الا ما جعلتو زهلا وانت تجل الحزن اذا شئت سهلا اللهم اعن على ذكرك وشكرك وحسن عبادتك يا رب الكريم It's absolutely fantastic to see you all here. Alhamdulillah. My apologies to all the folks who are both here and online. Standard obviously, technical issues always on the first lesson. Obviously new network, new planning, new ideas, such grand plans, none of it's come off. We're meant to have this screen on, that screen on, we're meant to have all kinds of bells and whistles. And have a look at all this, by the way, you guys can see all this, this huge mega setup. Where's gwrs, byddwch yn gallu gweld yr holl gwrs. Mae'r set-up hwnnw'n mawr, mega. Pa lle mae Yahya wedi mynd, ar ôl hynny? Pa lle yw Yahya? Yn y gwaelod. Yn y gwaelod. Yahya, dyna ddewiswch, dyna dewiswch i Rakwan dewiswch dewiswch ffotograffau yma o'r holl beth hwn, ia? Ac mae'r holl beth wedi'i 